### Importing Libraries

In [3]:
import osgeo
from osgeo import gdal
import rasterio
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
for gpu in gpus:
    print(gpu)
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
import re
    
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU,  Dense, Flatten, Reshape, Dropout, UpSampling2D, Lambda
from tensorflow.keras.preprocessing import image
from PIL import Image
import os

In [5]:
file_path = "images_downscaled\\April_7_2022.tif"
dataset = rasterio.open(file_path)

# Print general information
print("Raster Information:")
print(f"Number of Bands: {dataset.count}")
print(f"Width: {dataset.width}")
print(f"Height: {dataset.height}")
print(f"Coordinate Reference System (CRS): {dataset.crs}")
print(f"Transform (Affine): {dataset.transform}")

Raster Information:
Number of Bands: 4
Width: 256
Height: 256
Coordinate Reference System (CRS): None
Transform (Affine): | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


c:\Users\remne\AppData\Local\Programs\Python\Python311\Lib\site-packages\rasterio\__init__.py:319: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Downscaling the Images to a (256, 256) Dimension.

In [3]:
input_folder = "images"
output_folder = "images_downscaled"
target_resolution = (256, 256)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over the files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is an image (you can modify the condition based on your image file extensions)
    if file_name.endswith(".tif"):
        # Construct the full paths to the input and output images
        input_image_path = os.path.join(input_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name)

        # Open and resize the input image
        input_image = Image.open(input_image_path)
        resized_image = input_image.resize(target_resolution)

        # Save the resized image to the output folder
        resized_image.save(output_image_path)

Downscaling the Masks to a (256, 256) Dimension.

In [4]:
input_folder = "masks"
output_folder = "masks_downscaled"
target_resolution = (256, 256)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over the files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is an image (you can modify the condition based on your image file extensions)
    if file_name.endswith(".tif"):
        # Construct the full paths to the input and output images
        input_image_path = os.path.join(input_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name)

        # Open and resize the input image
        input_image = Image.open(input_image_path)
        resized_image = input_image.resize(target_resolution)

        # Save the resized image to the output folder
        resized_image.save(output_image_path)

In [2]:
def build_generator():

    model = Sequential()

    model.add(Dense(256*256, input_dim = 100))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((256, 256, 1))) # 1 is the number of channels

    # Upsampling Block 1
    model.add(UpSampling2D())
    model.add(Conv2D(64, 3, padding='same'))
    model.add(LeakyReLU(0.2))

    # Upsampling Block 2
    model.add(UpSampling2D())
    model.add(Conv2D(64, 3, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional Block 1
    model.add(Conv2D(64, 2, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutuonal Block 2
    model.add(Conv2D(64, 2, padding ='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional layer to get to one channel
    model.add(Conv2D(1, 1, padding='same', activation='sigmoid'))


    return model

generator = build_generator()

generator.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 65536)             6619136   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 65536)             0         
                                                                 
 reshape (Reshape)           (None, 256, 256, 1)       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 512, 512, 1)      0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 64)      640       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512, 512, 64)      0         
                                                        

In [55]:
def build_discriminator():

    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=(256, 256, 1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Convolutional Block 2
    model.add(Conv2D(128, (3, 3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Convolutional Block 3
    model.add(Conv2D(256, (3, 3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Convolutional Block 4
    model.add(Conv2D(512, (3, 3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Flatten and then Condense into 1
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [56]:
discriminator = build_discriminator()

In [57]:
discriminator.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_47 (Conv2D)          (None, 127, 127, 64)      640       
                                                                 
 leaky_re_lu_46 (LeakyReLU)  (None, 127, 127, 64)      0         
                                                                 
 dropout_53 (Dropout)        (None, 127, 127, 64)      0         
                                                                 
 conv2d_48 (Conv2D)          (None, 125, 125, 128)     73856     
                                                                 
 leaky_re_lu_47 (LeakyReLU)  (None, 125, 125, 128)     0         
                                                                 
 dropout_54 (Dropout)        (None, 125, 125, 128)     0         
                                                                 
 conv2d_49 (Conv2D)          (None, 123, 123, 256)   